# 🌐 AgentCore Browser with Web Auth Signing 

## Overview

In this tutorial we will learn how to enable Web Bot Auth signing with Amazon Bedrock AgentCore Browser tool. 

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands                                                                          |
| LLM model           | Claude 4.5 Haiku                                                                 |
| Tutorial components | Browser automation, Web Bot Auth signing of browser Requests                     |
| Tutorial vertical   |                                                                                  |
| Example complexity  | Intermediate                                                                     |
| SDK used            | Amazon BedrockAgentCore Python SDK, Strands Agents, Strands Agent Tools          |
 
### Tutorial Architecture


### Tutorial Key Features

* Using browser tool in a headless way with Strands agents
* AgentCoreBrowser integration using strands_tools.browser
* Claude 4.5 Haiku model for fast and efficient analysis

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* Strands agents and strands-agents-tools packages
* Access to Claude 4.5 Haiku model in Amazon Bedrock
* AgentCore role for bot session recording capabilities

**Implementation**: Using the official `strands_tools.browser.AgentCoreBrowser` for seamless browser automation with intelligent financial analysis.

## 🎯 What This Does
- **AgentCoreBrowser Integration** using `strands_tools.browser`
- **Structured Invoke Pattern** for implementations
- **Claude 4.5 Haiku Model** for fast and efficient analysis

## 🔧 AgentCore Browser Configuration

Before using the AgentCore Browser tool, you can optionally create a custom browser configuration with specific settings like recording capabilities, network configuration, and execution roles. This section shows how to create a custom browser configuration using the AWS SDK.

### Create Custom Browser Configuration

The following code demonstrates how to create a custom AgentCore Browser with recording enabled and a specific execution role:

#### 📚 Create the Browser Configuration with Web Bot Auth Signing

**What is Browser Signing?**

Browser signing (`browserSigning.enabled = True`) configures the AgentCore Browser to automatically sign all outgoing HTTP requests. This is essential for:

- **Authenticated API access**: Sign requests to protected APIs
- **AWS service integration**: Automatically sign requests with AWS credentials
- **Security compliance**: Ensure all browser requests are authenticated
- **Request integrity**: Cryptographically sign request headers

When enabled, the browser will:
1. Intercept all HTTP/HTTPS requests
2. Add cryptographic signatures to request headers
3. Include authentication tokens automatically
4. Maintain session integrity across requests


In [ ]:
import boto3
import uuid
import os
import sys

tutorials_path = os.path.abspath(os.path.join(os.getcwd(), '../../../'))
if tutorials_path not in sys.path:
    sys.path.insert(0, tutorials_path)

from utils import create_agentcore_role

cp_client = boto3.client('bedrock-agentcore-control', 
                         region_name='us-west-2')

dp_client = boto3.client('bedrock-agentcore',
                         region_name='us-west-2')

accountId = boto3.client("sts").get_caller_identity()["Account"] 
region = boto3.Session().region_name

print(f"Account ID: {accountId}")
print(f"Region: {region}")

execution_role_arn = create_agentcore_role("web-bot-auth")["Role"]["Arn"]

print(f"\n✅ Role Created Successfully : {execution_role_arn}")

response = cp_client.create_browser(
    name="web_bot_auth_browser_" + str(uuid.uuid4())[:6],
    description="Browser configured to sign web bot auth",
    networkConfiguration={
        "networkMode": "PUBLIC"
    },
    executionRoleArn=execution_role_arn,
    browserSigning={
        "enabled": True
    }
)

browserId = response['browserId']
browserArn = response['browserArn']
print(f"\n✅ Browser Created Successfully!")
print(f"   Browser ID: {browserId}")
print(f"   Browser ARN: {browserArn}")
print(f"\n🔐 Browser signing is ENABLED - all requests will be automatically signed")

In [ ]:
from strands import Agent
from strands_tools.browser import AgentCoreBrowser
import asyncio
import time

print("🚀 Using native async/await - no nest_asyncio needed!")

# Create and configure the Strands agent with AgentCoreBrowser
# Initialize the official AgentCoreBrowser tool with custom browser ID
agent_core_browser = AgentCoreBrowser(identifier=browserId, region="us-west-2")
    
# Create agent with Claude 4.5 Haiku model and optimized settings
strands_agent = Agent(
    tools=[agent_core_browser.browser],
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="""You are a website analyst with browser signing capabilities.
1. Use the browser tool to visit and interact with the website EFFICIENTLY
2. Focus on extracting key information QUICKLY and within 2-3 browser interactions.
3. All requests will be signed automatically via browser signing configuration."""
)

print("\n🚀 Starting analysis with browser signing enabled...")

# Define async function
async def analyze_website(agent, prompt):
    """Async wrapper for agent invocation"""
    try:
        # Use the async method directly
        result = await agent.invoke_async(prompt)
        return result
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return None
    
# Use top-level await (works natively in Jupyter/IPython 7.0+)
result = await analyze_website(
    strands_agent,
    "Please analyze the website at https://www.amazon.com/ and provide key insights"
)

if result:
    print("\n✅ Analysis completed!")
    print(result)

## 🎭 What Happened Behind the Scene

When you execute this notebook with **browser signing enabled**, the following process occurs:

### 1. **Browser Configuration Creation**
```python
browserSigning={
    "enabled": True
}
```
This tells the AgentCore Browser service to automatically sign all HTTP requests made by the browser.

### 2. **Agent Initialization**
The Strands agent is initialized with:
- Custom browser ID (with signing enabled)
- Claude 4.5 Haiku model
- Browser tool integration

### 3. **Request Signing Flow**
When the agent navigates to websites:

```
Agent Request → AgentCore Browser → Sign Request Headers → Target Website
                    ↓
            Add Authentication Headers:
            - Authorization signatures
            - Timestamp headers
            - Security tokens
            - Request checksums
```

## 🔒 Security Benefits

With browser signing enabled:

✅ **Automatic Authentication** - No manual header management  
✅ **Request Integrity** - Cryptographic signatures prevent tampering  
✅ **AWS Integration** - Seamless IAM credential integration  
✅ **Session Management** - Secure session token handling  
✅ **Audit Trail** - All signed requests are logged  

## 🛠️ Troubleshooting

### Error: "Browser session not found"
**Solution**: The browser ID might have expired. Re-run the browser creation cell (cell-3)
